# Solar ZED
## Predict energy produced by solar panels

# 1. Wprowadzenie

The goal of the is to predict the energy produced by solar panels. The value to be predicted concerns hourly energy production denoted in kWh. Task is to create an algorithm that predicts this value based on 49 descriptive attributes.

To create a predictive model, students should use Python and scikit-learn. 

# 2. Importy

In [212]:
import pandas as pd
from sklearn import datasets
import sklearn.model_selection as skms
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import mean_squared_error
from math import sqrt

# 3. Wczytanie danych treniningowych z pliku

In [386]:
train_df = pd.read_csv('train.csv', sep=',')
test_df = pd.read_csv('test.csv', sep=',')

In [387]:
pd.options.display.max_columns = None
train_df.head()

,id,idsito,idmodel,idbrand,lat,lon,ageinmonths,anno,day,ora,data,temperatura_ambiente,irradiamento,pressure,windspeed,humidity,icon,dewpoint,windbearing,cloudcover,tempi,irri,pressurei,windspeedi,humidityi,dewpointi,windbearingi,cloudcoveri,dist,altitude,azimuth,altitudei,azimuthi,pcnm1,pcnm2,pcnm3,pcnm4,pcnm5,pcnm6,pcnm7,pcnm8,pcnm9,pcnm10,pcnm11,pcnm12,pcnm13,pcnm14,pcnm15,irr_pvgis_mod,irri_pvgis_mod,kwh
0,97595,0.425,0.083,0.083,0.437,0.631,1.000,2012,0.003,0.0,1/2/2012 2:00,0.212,0.0,0.0,0.054,0.81,0.083,0.436,0.557,0.0,0.113,0.219,0.0,0.040,0.085,0.114,0.422,0.200,0.983607,0.118,0.148,0.520,0.813,0.376,0.095,1.000,0.438,1.000,0.000,0.011,0.127,0.599,0.553,0.390,0.828,0.562,0.485,0.612,0.0,0.189,0.0
1,97606,0.250,0.208,0.083,0.437,0.620,0.063,2012,0.003,0.0,1/2/2012 2:00,0.212,0.0,0.0,0.062,0.80,0.083,0.444,0.585,0.0,0.117,0.220,0.0,0.038,0.046,0.188,0.374,0.329,0.983607,0.117,0.147,0.508,0.780,0.380,0.526,0.747,0.534,0.403,0.765,0.061,0.516,0.867,0.000,0.086,0.990,0.841,0.606,0.616,0.0,0.200,0.0
2,97637,0.350,0.375,0.000,0.439,0.626,0.000,2012,0.003,0.0,1/2/2012 2:00,0.212,0.0,0.0,0.054,0.76,0.083,0.432,0.557,0.0,0.115,0.220,0.0,0.039,0.044,0.201,0.336,0.201,0.983607,0.118,0.147,0.520,0.825,0.378,0.348,0.000,0.314,0.583,0.465,0.040,0.000,0.072,0.432,0.257,0.884,0.597,0.539,0.614,0.0,0.158,0.0
3,97646,0.325,0.292,0.167,0.440,0.612,0.250,2012,0.003,0.0,1/2/2012 2:00,0.212,0.0,0.0,0.060,0.76,0.083,0.442,0.578,0.0,0.115,0.220,0.0,0.040,0.044,0.170,0.351,0.198,0.983607,0.118,0.145,0.507,0.782,0.384,0.972,0.485,0.360,0.462,0.358,0.112,0.154,1.000,1.000,0.252,0.787,1.000,0.432,0.620,0.0,0.204,0.0
4,97772,0.000,0.000,0.333,0.436,0.626,0.719,2012,0.003,0.0,1/2/2012 2:00,0.212,0.0,0.0,0.054,0.82,0.083,0.436,0.557,0.0,0.119,0.220,0.0,0.038,0.046,0.114,0.422,0.320,0.983607,0.117,0.147,0.504,0.780,0.378,0.250,0.605,0.531,0.427,0.493,1.000,0.412,0.532,0.619,0.327,0.748,0.614,0.473,0.614,0.0,0.199,0.0


In [388]:
train_df.shape

(181603, 51)

In [389]:
test_df.shape

(54187, 50)

In [390]:
train_df.describe()

,id,idsito,idmodel,idbrand,lat,lon,ageinmonths,anno,day,ora,temperatura_ambiente,irradiamento,pressure,windspeed,humidity,icon,dewpoint,windbearing,cloudcover,tempi,irri,pressurei,windspeedi,humidityi,dewpointi,windbearingi,cloudcoveri,dist,altitude,azimuth,altitudei,azimuthi,pcnm1,pcnm2,pcnm3,pcnm4,pcnm5,pcnm6,pcnm7,pcnm8,pcnm9,pcnm10,pcnm11,pcnm12,pcnm13,pcnm14,pcnm15,irr_pvgis_mod,irri_pvgis_mod,kwh
count,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000,181603.000000
mean,154061.584368,0.212504,0.239457,0.152576,0.448749,0.574158,0.301208,2012.350705,0.415888,0.489254,0.357676,0.112929,0.654256,0.079732,0.683789,0.498017,0.591404,0.459134,0.345873,0.124625,0.222636,0.000245,0.038594,0.064790,0.119847,0.345554,0.205266,0.489598,0.541485,0.446112,0.208971,0.372708,0.420935,0.349307,0.601875,0.518272,0.415561,0.499350,0.111288,0.407170,0.542918,0.629529,0.333176,0.758798,0.645844,0.488550,0.572263,0.180267,0.199450,0.167387
std,77880.280535,0.131447,0.169260,0.116226,0.037412,0.149998,0.375275,0.477192,0.269510,0.303695,0.164667,0.136825,0.253942,0.051732,0.179720,0.294961,0.096611,0.226229,0.245143,0.067321,0.012773,0.004397,0.004408,0.044351,0.014517,0.035397,0.029121,0.294608,0.178835,0.195547,0.151439,0.187500,0.200182,0.214131,0.215377,0.256412,0.252444,0.239749,0.220751,0.263052,0.259418,0.209076,0.214232,0.228070,0.202587,0.181275,0.200393,0.223757,0.057927,0.208760
min,1.000000,0.000000,0.000000,0.000000,0.415000,0.154000,0.000000,2012.000000,0.000000,0.000000,0.045000,0.000000,0.000000,0.000000,0.160000,0.000000,0.139000,0.000000,0.000000,0.009000,0.108000,0.000000,0.000000,0.034000,0.063000,0.000000,0.000000,0.000000,0.111000,0.128000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.137000,0.000000,0.000000,0.000000,-0.025000,0.000000
25%,104972.500000,0.100000,0.167000,0.083000,0.437000,0.620000,0.000000,2012.000000,0.195000,0.222000,0.212000,0.000000,0.748000,0.045000,0.540000,0.083000,0.517000,0.313000,0.220000,0.073000,0.217000,0.000000,0.037000,0.044000,0.114000,0.336000,0.196000,0.224044,0.414000,0.289000,0.096000,0.210000,0.377000,0.250000,0.551000,0.363000,0.331000,0.339000,0.031000,0.204000,0.527000,0.553000,0.257000,0.748000,0.614000,0.432000,0.612000,0.000000,0.158000,0.000000
50%,160724.000000,0.225000,0.208000,0.167000,0.437000,0.624000,0.125000,2012.000000,0.386000,0.500000,0.333000,0.040000,0.753000,0.071000,0.700000,0.667000,0.595000,0.488000,0.310000,0.112000,0.220000,0.000000,0.038000,0.044000,0.114000,0.336000,0.196000,0.480874,0.560000,0.411000,0.139000,0.297000,0.378000,0.377000,0.605000,0.531000,0.427000,0.493000,0.052000,0.412000,0.532000,0.619000,0.327000,0.760000,0.614000,0.473000,0.614000,0.059000,0.196000,0.049000
75%,218611.500000,0.325000,0.292000,0.167000,0.439000,0.630000,0.719000,2013.000000,0.592000,0.778000,0.485000,0.210000,0.755000,0.106000,0.840000,0.667000,0.668000,0.662000,0.480000,0.126000,0.222000,0.000000,0.039000,0.063000,0.118000,0.339000,0.198000,0.748634,0.676000,0.622000,0.273000,0.492000,0.380000,0.422000,0.730000,0.634000,0.462000,0.493000,0.114000,0.511000,0.600000,0.717000,0.390000,0.884000,0.738000,0.530000,0.615000,0.332000,0.216000,0.331000
max,276488.000000,0.425000,0.750000,0.417000,0.553000,0.691000,1.000000,2013.000000,1.000000,1.000000,0.773000,0.710000,0.7

Korelacja między atrybutem decyzyjnym (kWh), a pozostałymi:

In [404]:
corrmatrix = train_df.corr()
corrmatrix['kwh'].sort_values(ascending=False)

kwh                     1.000000
irradiamento            0.874104
irr_pvgis_mod           0.847748
irri                    0.437936
altitude                0.412323
irri_pvgis_mod          0.271077
tempi                   0.254622
temperatura_ambiente    0.221136
id                      0.150664
humidityi               0.133538
windbearingi            0.106745
dewpoint                0.106080
windspeed               0.100720
pressure                0.055421
anno                    0.034170
pcnm9                   0.020891
pcnm6                   0.018604
day                     0.017165
pcnm13                  0.016048
pcnm2                   0.015804
idsito                  0.014446
pcnm8                   0.012128
idmodel                 0.008186
lon                     0.006655
dewpointi               0.005425
pcnm11                  0.004418
pcnm1                   0.003846
pcnm12                  0.000650
pcnm14                  0.000361
pcnm15                  0.000157
pcnm3     

# 4. Preprocessing danych

Przeniesienie atrybutu decyzyjnego (kwh) do osobnego data frame:

In [391]:
train_df_kwh = train_df.loc[:,'kwh']

train_df_kwh.tail()

181598    0.511
181599    0.511
181600    0.511
181601    0.511
181602    0.511
Name: kwh, dtype: float64

Usunięcie zbędnych kolumn ze zbioru treningowego (id, data, skopiowane wyżej kwh)

In [392]:
train_df_clean = train_df.drop(columns=['kwh', 'data', 'id'], axis = 1)

Analogicznie dla zbioru testowego:

In [393]:
test_df_id = test_df.loc[:,'id']
test_df_clean = test_df.drop(columns=['data', 'id'], axis = 1)

Wartości kolumn będącacymi identyfikatorami modelu/marki, jako osobne kolumny:

In [394]:
train_df_clean = pd.get_dummies(train_df_clean, prefix=['idsito', 'idmodel', 'idbrand'], columns=['idsito', 'idmodel', 'idbrand'])
test_df_clean = pd.get_dummies(test_df_clean, prefix=['idsito', 'idmodel', 'idbrand'], columns=['idsito', 'idmodel', 'idbrand'])

In [395]:
train_df_clean.iloc[0:5, 44:]

,irri_pvgis_mod,idsito_0.0,idsito_0.025,idsito_0.05,idsito_0.075,idsito_0.1,idsito_0.125,idsito_0.15,idsito_0.2,idsito_0.225,idsito_0.25,idsito_0.275,idsito_0.3,idsito_0.325,idsito_0.35,idsito_0.375,idsito_0.4,idsito_0.425,idmodel_0.0,idmodel_0.083,idmodel_0.125,idmodel_0.16699999999999998,idmodel_0.20800000000000002,idmodel_0.25,idmodel_0.292,idmodel_0.375,idmodel_0.41700000000000004,idmodel_0.45799999999999996,idmodel_0.75,idbrand_0.0,idbrand_0.083,idbrand_0.16699999999999998,idbrand_0.25,idbrand_0.33299999999999996,idbrand_0.41700000000000004
0,0.189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0.200,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,0.158,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,0.204,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,0.199,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Filtrowanie kolumn do tych, o ważności przynajmniej 0.001 (lista utworzona w ostatnim wierszu działu 5 - jeżeli tam ma zwrócić wiersze, nie uruchamiać poniższej linii):

In [396]:
columns_to_filter = ['irr_pvgis_mod', 'irradiamento', 'irri', 'irri_pvgis_mod', 'dist', 'humidity', 'cloudcover', 'day', 'anno', 'tempi', 'azimuth', 'dewpoint', 'azimuthi', 'pressure', 'ageinmonths', 'windbearing', 'windspeed', 'altitude', 'temperatura_ambiente', 'altitudei', 'pcnm7', 'windbearingi', 'ora', 'lon', 'cloudcoveri', 'humidityi', 'windspeedi', 'dewpointi', 'pcnm3']
train_df_clean = train_df_clean.loc[:,train_df_clean.columns.str.contains('|'.join(columns_to_filter))]
test_df_clean = test_df_clean.loc[:,test_df_clean.columns.str.contains('|'.join(columns_to_filter))]

In [397]:
train_df_clean.shape

(181603, 30)

Normalizacja danych:

In [398]:
def normalize(df):
    x = df.values #numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
    return df

In [399]:
train_df_clean = normalize(train_df_clean)
test_df_clean = normalize(test_df_clean)

In [400]:
train_df_clean.head()

,lon,ageinmonths,anno,day,ora,temperatura_ambiente,irradiamento,pressure,windspeed,humidity,dewpoint,windbearing,cloudcover,tempi,irri,pressurei,windspeedi,humidityi,dewpointi,windbearingi,cloudcoveri,dist,altitude,azimuth,altitudei,azimuthi,pcnm3,pcnm7,irr_pvgis_mod,irri_pvgis_mod
0,0.888268,1.000,0.0,0.003,0.0,0.229396,0.0,0.0,0.077586,0.773810,0.409091,0.724317,0.0,0.106776,0.124439,0.0,0.133333,0.093578,0.144886,0.422,0.200,0.983607,0.009056,0.028986,0.529532,0.813,1.000,0.011,0.0,0.207565
1,0.867784,0.063,0.0,0.003,0.0,0.229396,0.0,0.0,0.089080,0.761905,0.420110,0.760728,0.0,0.110883,0.125561,0.0,0.126667,0.022018,0.355114,0.374,0.329,0.983607,0.007762,0.027536,0.517312,0.780,0.747,0.061,0.0,0.218235
2,0.878957,0.000,0.0,0.003,0.0,0.229396,0.0,0.0,0.077586,0.714286,0.403581,0.724317,0.0,0.108830,0.125561,0.0,0.130000,0.018349,0.392045,0.336,0.201,0.983607,0.009056,0.027536,0.529532,0.825,0.000,0.040,0.0,0.177498
3,0.852886,0.250,0.0,0.003,0.0,0.229396,0.0,0.0,0.086207,0.714286,0.417355,0.751625,0.0,0.108830,0.125561,0.0,0.133333,0.018349,0.303977,0.351,0.198,0.983607,0.009056,0.024638,0.516293,0.782,0.485,0.112,0.0,0.222114
4,0.878957,0.719,0.0,0.003,0.0,0.229396,0.0,0.0,0.077586,0.785714,0.409091,0.724317,0.0,0.112936,0.125561,0.0,0.126667,0.022018,0.144886,0.422,0.320,0.983607,0.007762,0.027536,0.513238,0.780,0.605,1.000,0.0,0.217265


# 5. Uczenie

In [419]:
#Empirical good default values are max_features=n_features for regression problems
tuned_parameters = [{'max_features': [28,29]}]

In [406]:
X_train, X_test, y_train, y_test = skms.train_test_split(train_df_clean, train_df_kwh, test_size=0.3, random_state=0)

In [420]:
#neg_mean_squared_error to minimise error (https://github.com/scikit-learn/scikit-learn/issues/2439)
clf = skms.GridSearchCV(RandomForestRegressor(n_estimators=125, n_jobs = 2), tuned_parameters, cv=5, scoring='neg_mean_squared_error')

In [421]:
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=125, n_jobs=2,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_features': [28, 29]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [422]:
print(clf.best_params_) 

{'max_features': 28}


Sprawdzenie znaczenia dla każdego atrybutu:

In [380]:
importances = list(clf.best_estimator_.feature_importances_)

In [381]:
columns = list(train_df_clean.columns)
feature_importances = [(train_df_clean, round(importance, 4)) for train_df_clean, importance in zip(columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
feature_importances

[('irradiamento', 0.7339),
 ('irr_pvgis_mod', 0.0938),
 ('humidity', 0.0247),
 ('irri_pvgis_mod', 0.0216),
 ('dist', 0.0206),
 ('cloudcover', 0.0195),
 ('day', 0.0151),
 ('azimuth', 0.0146),
 ('irri', 0.0119),
 ('dewpoint', 0.0104),
 ('anno', 0.0096),
 ('tempi', 0.0075),
 ('azimuthi', 0.007),
 ('cloudcoveri', 0.0036),
 ('humidityi', 0.0035),
 ('dewpointi', 0.0028)]

In [307]:
#0.001
feature_importances_filtered = filter(lambda x: x[1] > 0.001, feature_importances)
result_dict = dict(feature_importances_filtered)
result_dict.keys()
#lista zawierająca atrybuty o ważności większej niż 0.001

dict_keys(['irr_pvgis_mod', 'irradiamento', 'irri', 'irri_pvgis_mod', 'dist', 'humidity', 'cloudcover', 'day', 'anno', 'tempi', 'azimuth', 'dewpoint', 'azimuthi', 'pressure', 'ageinmonths', 'windbearing', 'windspeed', 'altitude', 'temperatura_ambiente', 'altitudei', 'pcnm7', 'windbearingi', 'ora', 'lon', 'cloudcoveri', 'humidityi', 'windspeedi', 'dewpointi', 'pcnm3'])

# 6. Testowanie modelu

In [423]:
y_true, y_pred = y_test, clf.predict(X_test)

RMSE:

In [424]:
sqrt(-clf.cv_results_['mean_test_score'][0])
#0.04935996749627618

0.04935996749627618

# 7. Wyniki dla zbioru testowego

In [425]:
result = clf.predict(test_df_clean)

In [426]:
result

array([5.79272000e-01, 5.46568000e-01, 5.46568000e-01, ...,
       2.20449909e-04, 1.08012809e-02, 5.49872565e-04])

In [427]:
test_df_id.head()

0    75823
1    75825
2    75858
3    75860
4    75906
Name: id, dtype: int64

In [428]:
ids = test_df_id.values
ids

array([ 75823,  75825,  75858, ...,  30799,  33064, 147846], dtype=int64)

Przygotownie wyniku:

In [429]:
result_with_id = {'Id': ids, 'Predicted': result }

from_dict = pd.DataFrame.from_dict(result_with_id)
from_dict.head()

,Id,Predicted
0,75823,0.579272
1,75825,0.546568
2,75858,0.546568
3,75860,0.546568
4,75906,0.579272


Zapis wyniku do pliku csv:

In [430]:
from_dict.to_csv('result.csv', encoding='utf-8', index=False)